# API Exercises

Here are two exercises that involve working with APIs and dictionaries.

One is using the Open Brewery API found at https://www.openbrewerydb.org/, and the other is using the API for UK Police Data, found at https://data.police.uk/docs/.

You can complete them in either order!

Remember that you can create new cells with esc + a or b

## Breweries

### Q1: Load the first page of results with 50 results per page

In [1]:
import requests
url = 'https://api.openbrewerydb.org/breweries'

search_parameters = {'page':1, 'per_page':50}

resp1 = requests.get('https://api.openbrewerydb.org/breweries', params = search_parameters)

In [2]:
print(type(resp1))
print(resp1.url)
print(resp1)


<class 'requests.models.Response'>
https://api.openbrewerydb.org/breweries?page=1&per_page=50
<Response [200]>


### Q2: This is only the first 50 results.  Get the next 50 and put them together.

In [3]:
search_parameters = {'page':2, 'per_page':50}

resp2 = requests.get('https://api.openbrewerydb.org/breweries', params = search_parameters)


In [4]:
print(type(resp2))
print(resp2.url)
print(resp2.status_code)

<class 'requests.models.Response'>
https://api.openbrewerydb.org/breweries?page=2&per_page=50
200


In [5]:
concat_list = resp1.json()

In [6]:
for i in range(len(resp2.json())):
    concat_list.append(resp2.json()[i])
    

In [7]:
print(len(concat_list))

100


### Q3: How many of these 100 breweries in are in Alaska?

In [8]:
breweries_in_alaska = 0
for i in range(len(concat_list)):
    if concat_list[i]['state'] == 'Alaska':
        breweries_in_alaska += 1
               


In [9]:
print(breweries_in_alaska)

3


In [10]:
concat_list[1].keys()

dict_keys(['id', 'name', 'brewery_type', 'street', 'city', 'state', 'postal_code', 'country', 'longitude', 'latitude', 'phone', 'website_url', 'updated_at', 'tag_list'])

### Q4: Of these 100 breweries, what are the different unique brewery types?

In [11]:
temp = [concat_list[i]['brewery_type'] for i in range(len(concat_list))]
unique_brewery_type = set(temp)
print(unique_brewery_type)

{'micro', 'contract', 'brewpub', 'planning', 'regional', 'proprietor'}


### Q5: What is the cloest brewery to "Devil's Potion Brewing Company LLC" ?
* Hint 1: Use Euclidian distance w/ longitude and latitude (assume longitude and latitude are a Carteisan coordinate system)
* Hint 2: You'll have to ignore the entries with "none" for latitude or longitude

In [12]:
import math    

def set_lat_lon(brewery):
    '''searches the concat_list to find an entry with a name matching the arg and returns lat and lon in a list'''
    temp2 = list(filter(lambda brew: concat_list[brew]['name'] == brewery, range(len(concat_list))))
    if concat_list[temp2[0]]['latitude']:
        start_brewery_lat = concat_list[temp2[0]]['latitude']
        start_brewer_lon = concat_list[temp2[0]]['longitude']
        origin = [float(start_brewery_lat), float(start_brewer_lon)]
    return origin
                                  
def distance(origin, destination):
    ''' determines the spherical distance between two sets of coordinates'''
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d
                   
def closest_bewery_to(start_brewery):
    '''takes a starting brewey and checks its distance from each brewery in a list\
    returns the distance of the closest brwery and all the information of that brewery'''
    origin = set_lat_lon(start_brewery)
    temp = []
    shortest_distance = 10000000
    for i in range(len(concat_list)):
        if concat_list[i]['name'] == start_brewery:
            pass
        elif concat_list[i]['latitude']:
            dest_brewery_lat = concat_list[i]['latitude']
            dest_brewery_lon = concat_list[i]['longitude']
            destination = [float(dest_brewery_lat), float(dest_brewery_lon)]
            current_distance = distance(origin, destination)
        else:
            pass
        
        if current_distance < shortest_distance:
            temp = concat_list[i]
            shortest_distance = current_distance
    return print('The closest brewery to {} is only {}km away!\n'.format(start_brewery, round(shortest_distance)) ), temp
                   
                   
    
print(closest_bewery_to("Devil's Potion Brewing Company LLC"))

The closest brewery to Devil's Potion Brewing Company LLC is only 7km away!

(None, {'id': 924, 'name': 'Port Brewing Co / The Lost Abbey', 'brewery_type': 'micro', 'street': '155 Mata Way Ste 104', 'city': 'San Marcos', 'state': 'California', 'postal_code': '92069-2983', 'country': 'United States', 'longitude': '-117.149141', 'latitude': '33.141537', 'phone': '8009186816', 'website_url': 'http://www.portbrewing.com', 'updated_at': '2018-08-24T00:01:07.397Z', 'tag_list': []})


### Q6: Write a function to find the closest brewery to any other given brewery

In [13]:
def bewery_a_to_b(start_brewery):
    origin = set_lat_lon(start_brewery)
    temp = []
    shortest_distance = 10000000
    for i in range(len(concat_list)):
        if concat_list[i]['name'] == start_brewery:
            pass
        elif concat_list[i]['latitude']:
            dest_brewery_lat = concat_list[i]['latitude']
            dest_brewery_lon = concat_list[i]['longitude']
            destination = [float(dest_brewery_lat), float(dest_brewery_lon)]
            current_distance = distance(origin, destination)
        else:
            pass
        
        if current_distance < shortest_distance:
            temp = concat_list[i]
            shortest_distance = current_distance
    return print('The closest brewery to {} is only {}km away!\n'.format(start_brewery, round(shortest_distance)) ), temp

### Q7: How would you get the first 10 pages from this API and put them all together using a for loop?

In [14]:
import requests
import time
url = 'https://api.openbrewerydb.org/breweries'

first_500_breweries = []
for i in range(0,9):
    page = 1
    search_parameters = {'page':page, 'per_page':50}
    temp_resp = requests.get('https://api.openbrewerydb.org/breweries', params = search_parameters)
    for i in range(len(temp_resp.json())):
        first_500_breweries.append(temp_resp.json()[i])
    page +=1
    time.sleep(5)

In [16]:
print(len(first_500_breweries))
type(first_500_breweries)

450


list

# Crime in the UK

### We will be analyzing different crimes reported in the UK as provided by https://data.police.uk/docs/

# Exploratory analysis
##### 1. How many total crimes were there at latitude : 52.63902 and -1.131321 on November of 2017.
Use the street level crimes data, the documentation for the API can be found at https://data.police.uk/docs/method/crime-street/

In [41]:
search = {'lat':52.63902, 'lng':-1.131321, 'date':'2017-11'}

crime = requests.get('https://data.police.uk/api/crimes-street/all-crime', params = search)



In [42]:
crime.status_code

200

In [43]:
print(len(crime.json()))

1334


In [45]:
print(crime.json()[1]['category'])

anti-social-behaviour


In [46]:
testing = [crime.json()[i]['category'] for i in range(len(crime.json()))]

In [47]:
d = {}
for category in testing:
    d[category] = d.get(category, 0) + 1

In [48]:
print(d)

{'anti-social-behaviour': 246, 'bicycle-theft': 81, 'burglary': 55, 'criminal-damage-arson': 66, 'drugs': 29, 'other-theft': 154, 'possession-of-weapons': 13, 'public-order': 73, 'robbery': 11, 'shoplifting': 176, 'theft-from-the-person': 41, 'vehicle-crime': 69, 'violent-crime': 301, 'other-crime': 19}


##### 2. We've queried the API once, but it could get annoying to retype the url over and over again, create a function `make_api_request` that enables you to query the API.


 The parameters for the function should be:
* lat (float) : latitude
* lng (float) : longitude
* date (string): Date in the format YYYY-MM
    * default value = `None`
    
And it should return a json object of 

for more information on default values check out http://blog.thedigitalcatonline.com/blog/2015/02/11/default-arguments-in-python/

In [29]:
def make_api_request(lat, lng, date = None):
    search = {'lat':float(lat), 'lng':float(lng), 'date':str(date)}
    r = requests.get('https://data.police.uk/api/crimes-street/all-crime', params = search)
    return r


##### 3. Write a function `categories_of_crime` that will determine the count of each type of crime for a given latitude and longitude. This is labelled as 'category' in the records. Your function should call the `make_api_request` function you created.

The parameters for the function should be:

* lat (float) : latitude
* lng (float) : longitude
* date (str) default = None

The function should return:
* a dictionary with the count of each type of crime



Once you've created the function, try it with these locations
* lat, lng of 51.5017861,-0.1432319   (Buckingham Palace)
* lat, lng of 53.480161, -2.245163     (Manchester)

In [40]:
def categories_of_crime (lat, lng, date = None):
    temp={}
    r = make_api_request(lat, lng, date)
    all_categories = [r.json()[i]['category'] for i in range(len(r.json()))]
    for category in all_categories:
        temp[category] = d.get(category, 0) + 1
    return temp
    
categories_of_crime(51.5017861,-0.1432319)

**Bonus**: 
* Write a function that determines the difference between Buckingham Palace and Manchester in terms of the number of crimes in each category.
    * In which category is there the largest absolute difference between the category of crime?
* Create a histogram depiction of the categories of crime

##### 4. Create a function `find_outcome_statuses` that will determine outcome statuses for a given latitude and longitude and date (optional)?
Investigate the data to determine where the outcome statuses are located.

**NOTE**: You'll notice that some of these crimes do not have crime outcomes. Make these into the category of "Not Resolved."

**NOTE 2**: These might take a long time to execute if you do not specify a month

**Bonus**: What is the ratio of crimes investigated to those not investigated? Is it higher near London or Manchester?

##### 5. Write a function `month_highest_crimes` that will return the month that had the highest number of crimes for a latitude, longitude and a year.

Inputs
* lat (float) : latitude
* lng (float) : longitude
* year (str) : in the format YYYY

Output
* month with highest crime (int)

**Bonus** Make a graph of how the number of crimes changed over time for a year. This will likely require a new function. Is seasonality a factor? Do the type of crimes change over time?

### Bonus Open Ended Questions

1. Take a look at the https://data.police.uk/docs/method/stops-street/ API. Is there a correlation between gender and being stopped and searched? How about race and being stopped and searched?